In [1]:
import telegram
import xml.etree.ElementTree as et
import json
import requests as req

In [2]:
from telegram import Update
from telegram.ext import Updater, CallbackContext
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters

## 미세먼지/코로나 현황 알리미 봇

### 미세먼지 정보 출력
* /reload 명령 입력시 관련정보를 서버에서 받아서 파일로 저장해 둠
* 이렇게 함으로써 사용자에게 나름 최신정보를 제공

### 서버에서 미세정보 파일 내려받기

In [3]:
# 서버에서 미세정보 파일 내려받기
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
returnType = 'json'
params = {'serviceKey':'GjgjTYLP10LmklhSDVDE6qQ8KZPc43DI+J0iaotwuQpc5cFzNY19Uo269sO/5MQJ+GT6AmPGbn3prlagN3nf7A==',
         'returnType':returnType,'sidoName':'전국','pageNo':1, 'numOfRows':1000, 'ver':'1.0'}
res = req.get(url,params=params)

In [4]:
# 내려받은 미세먼지 정보를 파일로 저장
with open('data/air.json','w', encoding='utf-8') as f:
    f.writelines(res.text)

### 지역정보 입력시 미세먼지 정보 출력

In [5]:
def pm10(val):
    grade = '😀'
    if val <= 30:
        grade = '😀'
    elif 30 < val <= 80:
        grade = '😐'
    elif 80 < val <= 150:
        grade = '😫'
    else:
        grade =  '😱'
        
    return grade

In [6]:
def pm25(val):
    grade = '😀'
    if val <= 15:
        grade =  '😀'
    elif 15 < val <= 35:
        grade = '😐'
    elif 35 < val <= 75:
        grade = '😫'
    else:
        grade = '😱'
    return grade
        

In [7]:
sido = '서울'
gu = '구로구'

with open('data/air.json', encoding='utf-8') as f:
    json_data = json.load(f)

# json에서 객체의 속성 호출 : 객체명['속성명'], 객체명.속성명
for item in json_data['response']['body']['items']:
    if (item['sidoName'] == sido) & (item['stationName'] == gu):
        pm10v = item['pm10Value']
        pm25v = item['pm25Value']
        print(item['sidoName'], item['stationName'], item['dataTime'], pm10v,pm10(int(pm10v)),pm25v,pm25(int(pm25v)))

서울 구로구 2022-11-24 12:00 64 😐 36 😫


### 미세먼지 챗봇 구현 (json)

In [8]:
# data.go.kr에서 대기정보 받아서 파일로 저장
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
returnType = 'json'
params = {'serviceKey':'GjgjTYLP10LmklhSDVDE6qQ8KZPc43DI+J0iaotwuQpc5cFzNY19Uo269sO/5MQJ+GT6AmPGbn3prlagN3nf7A==',
         'returnType':returnType,'sidoName':'전국','pageNo':1, 'numOfRows':1000, 'ver':'1.0'}
def reload_air():
    res = req.get(url,params=params)
    with open('data/air.json','w', encoding='utf-8') as f:
        f.writelines(res.text)    

In [9]:
# 특정지역의 대기정보와 이모지 받아오기
def fetch_air(sido,gu):
    with open('data/air.json', encoding='utf-8') as f:
        json_data = json.load(f)
        
    airinfo = '입력하신 지역에는 대기정보가 없어요!😂😂'
    for item in json_data['response']['body']['items']:
        if (item['sidoName'] == sido) & (item['stationName'] == gu):
            pm10v = item['pm10Value']
            pm25v = item['pm25Value']
            airinfo = f"🎉[{item['sidoName']} {item['stationName']} {item['dataTime']}]🎉 \n{pm10v}µg/m³ - {pm10(int(pm10v))} \n{pm25v}µg/m³ - {pm25(int(pm25v))}"
            break
            
    return airinfo

In [10]:
fetch_air('서울','구로구')

'🎉[서울 구로구 2022-11-24 12:00]🎉 \n64µg/m³ - 😐 \n36µg/m³ - 😫'

### 메세지 함수 정의

In [11]:
mykey = '5925683914:AAHodNMQ2UQgahXBusavtKQbyIjaQXy_Rsc'
chat_id = 5797700110

In [12]:
def air_message(update,context):
    info = update.message.text.split(',')
    airinfo = fetch_air(info[0].strip(),info[1].strip())
    context.bot.send_message(text=f'{airinfo}',
                   chat_id=update.message.chat_id)
    pass

In [13]:
def reload_message(update,context):
    reload_air()
    context.bot.send_message(text=f'🎉대기정보 갱신됨!!🎉',
                   chat_id=update.message.chat_id)


### 핸들러 등록

In [14]:
updater = Updater(mykey)
reload_handler = CommandHandler('reload',reload_message)
air_handler = MessageHandler(Filters.text,air_message)

updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(air_handler)

### 챗봇 서버 가동

In [15]:
bot = telegram.Bot(token=mykey)

print('챗봇 서버 가동중...')
updater.start_polling()
updater.idle()

챗봇 서버 가동중...
